In [23]:
from datasets import load_dataset
import os
from itertools import chain

In [14]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('ziqingyang/chinese-alpaca-2-7b')

'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /ziqingyang/chinese-alpaca-2-7b/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x124ec78d0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 7f5ff8cd-ff27-4e20-9465-150438037b7e)')' thrown while requesting HEAD https://huggingface.co/ziqingyang/chinese-alpaca-2-7b/resolve/main/tokenizer_config.json
You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [5]:
data_file = "../data/hdnj.txt"
cache_dir = "../cache"

In [6]:
raw_dataset = load_dataset("text", data_files=data_file, cache_dir=cache_dir, keep_in_memory=False)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 2181
    })
})

In [13]:
raw_dataset['train'][0]['text']

'第一章上古天真论篇'

In [15]:
def tokenize_function(examples):   
    output = tokenizer(examples["text"])
    return output

In [16]:
tokenized_dataset = raw_dataset.map(
    tokenize_function,
    batched=True,
    num_proc=8,
    remove_columns="text",
    load_from_cache_file=True,
    keep_in_memory=False,
    cache_file_names = {k: os.path.join(cache_dir, 'tokenized.arrow') for k in raw_dataset},
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset (num_proc=8):   0%|          | 0/2181 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2181
    })
})

In [22]:
block_size = 1024

In [47]:
chain(*tokenized_dataset['train'])

In [36]:
concatenated_examples = {k: list(chain(*tokenized_dataset[k])) for k in tokenized_dataset.keys()}

In [37]:
concatenated_examples

{'train': ['input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_ids',
  'attention_mask',
  'input_id

In [46]:
total_length = len(concatenated_examples[list(tokenized_dataset.keys())[0]])
total_length

4362

In [48]:
concatenated_examples.items()

dict_items([('train', ['input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'attention_mask', 'input_ids', 'at

In [ ]:
82528745

In [24]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [25]:
grouped_datasets = tokenized_dataset.map(
    group_texts,
    batched=True,
    num_proc=8,
    load_from_cache_file=True,
    keep_in_memory=False,
    cache_file_names = {k: os.path.join(cache_dir, 'grouped.arrow') for k in tokenized_dataset},
    desc=f"Grouping texts in chunks of {block_size}",
)

Grouping texts in chunks of 1024 (num_proc=8):   0%|          | 0/2181 [00:00<?, ? examples/s]

In [26]:
grouped_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 94
    })
})

In [34]:
print(grouped_datasets['train'][0]['labels'])

[1, 29871, 32141, 31374, 30429, 30960, 42338, 32758, 33211, 1, 1, 29871, 45566, 30505, 31491, 31456, 30214, 30486, 31325, 30648, 32761, 30214, 32871, 31325, 30815, 31243, 30214, 34691, 31325, 52952, 33055, 30214, 31143, 31325, 40892, 34005, 30214, 30494, 31325, 31451, 30408, 30267, 1, 29871, 35290, 31658, 30909, 30408, 32271, 35344, 30383, 32518, 33790, 30429, 30960, 42011, 30214, 40075, 33917, 30898, 31047, 32133, 30214, 31325, 33332, 30413, 37546, 31608, 31482, 30594, 42011, 30214, 30470, 32279, 31047, 31325, 33332, 33917, 37546, 30767, 30267, 30594, 30793, 32911, 34765, 30313, 30998, 31369, 30577, 34765, 30882, 1, 29871, 47510, 33178, 30783, 35344, 30383, 30429, 30960, 42011, 30882, 31149, 32282, 30767, 30214, 30545, 30909, 41067, 30214, 30503, 30909, 33908, 30354, 30214, 31855, 36529, 30417, 31669, 30214, 31558, 31924, 30417, 31190, 30214, 30413, 39800, 30732, 33939, 30214, 31969, 30815, 31305, 31267, 30648, 39430, 30214, 31325, 32840, 34076, 31149, 30408, 30470, 30214, 30898, 3104

In [49]:
grouped_datasets["train"].features.type

StructType(struct<input_ids: list<item: int32>, attention_mask: list<item: int8>, labels: list<item: int64>>)

In [52]:
tokenizer.decode(grouped_datasets['train'][0]['labels'])

'<s> 第一章上古天真论篇<s><s> 昔在黄帝，生而神灵，弱而能言，幼而徇齐，长而敦敏，成而登天。<s> 乃问于天师曰：余闻上古之人，春秋皆度百岁，而动作不衰；今时之人，年半百而动作皆衰者。时世异耶人将失之耶？<s> 岐伯对曰：上古之人？其知道者，法于阴阳，和于术数，食饮有节，起居有常，不妄作劳，故能形与神俱，而尽终其天年，度百岁乃去。<s> 今时之人不然也，以酒为浆，以妄为常，醉以入房，以欲竭其精，以耗散其真，不知持满，不时御神，务快其心，逆于生乐，起居无节，故半百而衰也。<s> 夫上古圣人之教下也，皆谓之虚邪贼风避之有时，恬惔虚无，真气从之，精神内守，病安从来。<s> 是以志闲而少欲，心安而不惧，形劳而不倦，气从以顺，各从其欲，皆得所愿。故美其食，任其服，乐其俗，高下不相慕，其民故曰朴。<s> 是以嗜欲不能劳其目，淫邪不能惑其心，愚智贤不肖，不惧于物，故合于道。<s> 所以能年皆度百岁而动作不衰者，以其德全不危也。<s> 帝曰：人年老而无子者，材力尽邪？将天数然也？<s> 岐伯曰：女子七岁肾气盛，齿更发长。<s> 二七而天癸至，任脉通，太冲脉盛，月事以时下，故有子。<s> 三七肾气平均，故真牙生而长极。<s> 四七筋骨坚，发长极，身体盛壮。<s> 五七阳明脉衰，面始焦，发始堕。<s> 六七三阳脉衰于上，面皆焦，发始白。<s> 七七任脉虚，太冲脉衰少，天癸竭，地道不通，故形坏而无子也。<s> 丈夫八岁肾气实，发长齿更。<s> 二八肾气盛，天癸至，精气溢泻，阴阳和，故能有子。<s> 三八肾气平均，筋骨劲强，故真牙生而长极。<s> 四八筋骨隆盛，肌肉满壮。<s> 五八肾气衰，发堕齿槁。<s> 六八阳气衰竭于上，面焦，发鬓颁白。<s> 七八肝气衰，筋不能动，天癸竭，精少，肾脏衰，形体皆极。<s> 八八则齿发去。<s> 肾者主水，受五脏六腑之精而藏之，故五脏盛，乃能泻。<s> 今五脏皆衰，筋骨解堕，天癸尽矣，故发鬓白，身体重，行步不正，而无子耳。<s> 帝曰：有其年已老，而有子者：何也？岐伯曰：此其天寿过度，气脉常通，而肾气有余也。此虽有子，男子不过尽八八，女子不过尽七七，而天地之精气皆竭矣。<s> 帝曰：夫道者年皆百岁，能有子乎？岐伯曰：夫道者能却老而全角，身年虽寿，能生子也。<s> 黄帝曰：余闻上古有真人者，提挈天地，把握阴阳，呼吸精气，独立守神，肌肉若一，故能寿

In [53]:
tokenizer.decode(grouped_datasets['train'][0]['input_ids'])

'<s> 第一章上古天真论篇<s><s> 昔在黄帝，生而神灵，弱而能言，幼而徇齐，长而敦敏，成而登天。<s> 乃问于天师曰：余闻上古之人，春秋皆度百岁，而动作不衰；今时之人，年半百而动作皆衰者。时世异耶人将失之耶？<s> 岐伯对曰：上古之人？其知道者，法于阴阳，和于术数，食饮有节，起居有常，不妄作劳，故能形与神俱，而尽终其天年，度百岁乃去。<s> 今时之人不然也，以酒为浆，以妄为常，醉以入房，以欲竭其精，以耗散其真，不知持满，不时御神，务快其心，逆于生乐，起居无节，故半百而衰也。<s> 夫上古圣人之教下也，皆谓之虚邪贼风避之有时，恬惔虚无，真气从之，精神内守，病安从来。<s> 是以志闲而少欲，心安而不惧，形劳而不倦，气从以顺，各从其欲，皆得所愿。故美其食，任其服，乐其俗，高下不相慕，其民故曰朴。<s> 是以嗜欲不能劳其目，淫邪不能惑其心，愚智贤不肖，不惧于物，故合于道。<s> 所以能年皆度百岁而动作不衰者，以其德全不危也。<s> 帝曰：人年老而无子者，材力尽邪？将天数然也？<s> 岐伯曰：女子七岁肾气盛，齿更发长。<s> 二七而天癸至，任脉通，太冲脉盛，月事以时下，故有子。<s> 三七肾气平均，故真牙生而长极。<s> 四七筋骨坚，发长极，身体盛壮。<s> 五七阳明脉衰，面始焦，发始堕。<s> 六七三阳脉衰于上，面皆焦，发始白。<s> 七七任脉虚，太冲脉衰少，天癸竭，地道不通，故形坏而无子也。<s> 丈夫八岁肾气实，发长齿更。<s> 二八肾气盛，天癸至，精气溢泻，阴阳和，故能有子。<s> 三八肾气平均，筋骨劲强，故真牙生而长极。<s> 四八筋骨隆盛，肌肉满壮。<s> 五八肾气衰，发堕齿槁。<s> 六八阳气衰竭于上，面焦，发鬓颁白。<s> 七八肝气衰，筋不能动，天癸竭，精少，肾脏衰，形体皆极。<s> 八八则齿发去。<s> 肾者主水，受五脏六腑之精而藏之，故五脏盛，乃能泻。<s> 今五脏皆衰，筋骨解堕，天癸尽矣，故发鬓白，身体重，行步不正，而无子耳。<s> 帝曰：有其年已老，而有子者：何也？岐伯曰：此其天寿过度，气脉常通，而肾气有余也。此虽有子，男子不过尽八八，女子不过尽七七，而天地之精气皆竭矣。<s> 帝曰：夫道者年皆百岁，能有子乎？岐伯曰：夫道者能却老而全角，身年虽寿，能生子也。<s> 黄帝曰：余闻上古有真人者，提挈天地，把握阴阳，呼吸精气，独立守神，肌肉若一，故能寿